Enable the `autoreload` extension, which will automatically reload the `playfab_manager` module when the source code is updated, using the following magic commands:

In [1]:
%load_ext autoreload
%autoreload 2

Import the `PlayFabManager` class from the `playfab_manager` module and create a `PlayFabManager` instance

In [2]:
from playfab_manager.playfab_manager import PlayFabManager
playfab = PlayFabManager()

Entity token is created


Use the `get_all_players` method of the `PlayFabManager` instance to retrieve a list of all players from the PlayFab:

In [3]:
# NOTE: players are already sorted by last login time
playfab.get_all_players()

Found 195 players


## Get traces

First, select recent players and get data for them

In [4]:
# Import the datetime module
import datetime

# Get the current time
current_time = datetime.datetime.now(datetime.timezone.utc)

# select the players in the last 24 hours
current_players = [p for p in playfab.all_players if current_time - p.LastLogin <= datetime.timedelta(hours=24)]

# print the last three player ids
[p.PlayerId for p in current_players[:4]]

['500334FD3F7DC28', '96EDF5118FBC01DB', '7F619B6752E2C434', '19C0A4BD953CD299']

Second, get files for each of the player

In [5]:
ids = [p.PlayerId for p in current_players[:4]]
playfab.download_player_files(ids)

Found 5 files for player 500334FD3F7DC28
Found 2 files for player 96EDF5118FBC01DB
Found 2 files for player 7F619B6752E2C434
Found 5 files for player 19C0A4BD953CD299


Merged files are stored in `TracesPandas`

In [6]:
playfab.all_players[0].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,9999,0,0,0,0,0
1,10099,0,0,349,0,0
2,10199,0,0,337,0,0
3,10299,0,0,325,0,0
4,10399,0,0,313,0,0


In [7]:
playfab.all_players[1].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,10839,0,-125,0,0,0
1,10939,0,-125,0,0,0
2,11039,0,-125,0,0,0
3,11139,0,-125,0,0,0
4,11239,0,-125,0,0,0


In [8]:
playfab.all_players[2].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,14439,120,0,0,0,0
1,14539,120,0,0,0,0
2,14639,120,0,0,0,0
3,14739,120,0,0,0,0
4,14839,120,0,0,0,0


In [9]:
playfab.all_players[3].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,29999,0,0,0,0,0
1,30099,0,0,351,0,0
2,30199,0,0,341,0,0
3,30299,0,0,331,0,0
4,30399,0,0,321,0,0


In [7]:
from utils.utils import merge_traces_dataframes
from visualization.animate_traces import animate_traces

In [8]:
resource = [p.TracesPandas.copy() for p in playfab.all_players[:4] if p.IsResource][0]
players = [p.TracesPandas.copy() for p in playfab.all_players[:4] if not p.IsResource]
len(players)

3

In [9]:
df_all = merge_traces_dataframes(players, resource)

In [10]:
df_all.head()

,timestamp_resource,x_resource,z_resource,rotation_resource,signaling_resource,score_resource,time,timestamp_1,x_1,z_1,...,z_2,rotation_2,signaling_2,score_2,timestamp_3,x_3,z_3,rotation_3,signaling_3,score_3
0,9999,0,0,0,0,0,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10099,0,0,349,0,0,10.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10199,0,0,337,0,0,10.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10299,0,0,325,0,0,10.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10399,0,0,313,0,0,10.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
server_id = playfab.all_players[0].ServerID
filename = server_id + "_traces.csv"
df_all.to_csv(filename, index=False)

In [12]:
df_all.fillna(method="bfill").head()

,timestamp_resource,x_resource,z_resource,rotation_resource,signaling_resource,score_resource,time,timestamp_1,x_1,z_1,...,z_2,rotation_2,signaling_2,score_2,timestamp_3,x_3,z_3,rotation_3,signaling_3,score_3
0,9999,0,0,0,0,0,10.0,20119.0,0.0,120.0,...,-125.0,0.0,0.0,0.0,14439.0,120.0,0.0,0.0,0.0,0.0
1,10099,0,0,349,0,0,10.1,20119.0,0.0,120.0,...,-125.0,0.0,0.0,0.0,14439.0,120.0,0.0,0.0,0.0,0.0
2,10199,0,0,337,0,0,10.2,20119.0,0.0,120.0,...,-125.0,0.0,0.0,0.0,14439.0,120.0,0.0,0.0,0.0,0.0
3,10299,0,0,325,0,0,10.3,20119.0,0.0,120.0,...,-125.0,0.0,0.0,0.0,14439.0,120.0,0.0,0.0,0.0,0.0
4,10399,0,0,313,0,0,10.4,20119.0,0.0,120.0,...,-125.0,0.0,0.0,0.0,14439.0,120.0,0.0,0.0,0.0,0.0


In [13]:
filename = server_id + "_traces.gif"
df_filled = df_all.fillna(method="bfill")

# valid_times = np.logical_and(df_filled.notna()["x_1"], df_filled.notna()["x_2"])
# valid_times = np.logical_and(valid_times, df_filled.notna()["x_3"])
# df_filled.loc[valid_times, :][:1000]

animate_traces(df_filled[:1000], n_players=len(players), save=True, filename=filename, fps=10)

In [14]:
animate_traces(df_filled, n_players=len(players), save=True, filename="fast_" + filename, fps=30)